In [1]:
import cv2
import numpy as np 
import sklearn 
import scipy
import matplotlib.pyplot as plt
import argparse
import math
import warnings
warnings.filterwarnings("ignore")

In [2]:
image = cv2.imread('302843992_485786496928626_6191234182030738228_n.png')

In [3]:
cv2.imshow('image',image)
cv2.waitKey(0)

-1

In [4]:
print("width: {} pixels".format(image.shape[1]))

width: 697 pixels


In [5]:
print("height: {} pixels".format(image.shape[0]))

height: 451 pixels


In [6]:
print("channels: {}".format(image.shape[2]))

channels: 3


In [7]:
image[0,0]

array([254, 254, 254], dtype=uint8)

In [3]:
def rgb2gray(image):
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            image[i][j]=0.3*image[i][j][0]+0.6*image[i][j][1]+0.1*image[i][j][2]
    return image

In [4]:
def rgb2graymax(image):
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            image[i][j]=max(image[i][j][0],image[i][j][1],image[i][j][2])
    return image

In [5]:
cv2.imshow('image',rgb2gray(image))
cv2.waitKey(0)

-1

In [11]:
cv2.imshow('image',rgb2graymax(image))
cv2.waitKey(0)

-1

In [12]:
grayscale = rgb2graymax(image)
cv2.imwrite("grayscale.png",grayscale)

True

In [6]:
image = cv2.imread('302843992_485786496928626_6191234182030738228_n.png')

In [7]:
def split_image(image):
    zeros = np.zeros(image.shape[:2], dtype="uint8")
    red_image = np.zeros(image.shape[:3], dtype="uint8")
    green_image = np.zeros(image.shape[:3], dtype="uint8")
    blue_image = np.zeros(image.shape[:3], dtype="uint8")
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if sum(image[i,j] >= np.array([200, 200, 200])) == 3:
                blue_image[i,j] = image[i,j]
                green_image[i,j] = image[i,j]
                red_image[i,j] = image[i,j]
            else:
                blue_image[i,j] = np.array([image[i,j,0],0,0])
                green_image[i,j] = np.array([0,image[i,j,1],0])
                red_image[i,j] = np.array([0,0,image[i,j,2]])
    
    return red_image,green_image,blue_image

In [8]:
red,green,blue = split_image(image)

In [9]:
cv2.imshow("red",red)
cv2.imshow("green",green)
cv2.imshow("blue",blue)
cv2.waitKey(0)


-1

In [28]:
canvas = np.zeros((300, 300, 3), dtype = "uint8")

In [24]:
cv2.imshow("canvas",canvas)
cv2.waitKey(0)

-1

In [18]:
def in_polygon(x, y, polygon):
    # check if the point (x,y) is inside the polygon without cv2
    result = False
    for i in range(len(polygon)):
        j = (i+1)%len(polygon)
        if (polygon[i][1] > y) != (polygon[j][1] > y) and \
            x < (polygon[j][0] - polygon[i][0]) * (y - polygon[i][1]) / (polygon[j][1] - polygon[i][1]) + polygon[i][0]:
            result = not result
    return result


def draw(canvas,polygon,color,offset_x=0,offset_y=0):
    canvas = canvas.copy()
    polygon = np.array(polygon)
    polygon[:,0] += offset_x
    polygon[:,1] += offset_y
    
    # each pixel in canvas 
    for i in range(canvas.shape[0]):
        for j in range(canvas.shape[1]):
            # check if the pixel is inside the polygon
            if in_polygon(j,i,polygon):
                canvas[i,j] = color
            
    return canvas

In [30]:
triangle = [(0,0),(30,0),(0,40)]
canvas = draw(canvas,triangle,(0,255,255),5,5)
cv2.imshow("canvas",canvas)
cv2.waitKey(0)


-1

In [31]:
triangle = [(10,10),(35,10),(22,31)]
canvas = draw(canvas,triangle,(0,0,255),40,5)
cv2.imshow("canvas",canvas)
cv2.waitKey(0)

-1

In [32]:
square = [(0,0),(50,0),(50,50),(0,50)]
canvas = draw(canvas,square,(128,128,128),5,40)
cv2.imshow("canvas",canvas)
cv2.waitKey(0)

-1

In [33]:
rhombus = [(0,0),(20,0),(0,34),(-20,34)]
canvas = draw(canvas,rhombus,(0,255,0),75,50)
cv2.imshow("canvas",canvas)
cv2.waitKey(0)

-1

In [10]:
image = cv2.imread('302843992_485786496928626_6191234182030738228_n.png')

In [12]:
M = np.float32([[1, 0, 25], [0, 1, 50]])

In [13]:
shifted = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
cv2.imshow("Shifted Down and Right", shifted)
cv2.waitKey(0)

-1

In [14]:
def shift_image(image, dx, dy):
    image = np.roll(image, dy, axis=0)
    image = np.roll(image, dx, axis=1)
    if dy>0:
        image[:dy, :] = 0
    elif dy<0:
        image[dy:, :] = 0
    if dx>0:
        image[:, :dx] = 0
    elif dx<0:
        image[:, dx:] = 0
    return image

In [15]:
cv2.imshow("Shifted Down and Right", shift_image(image, 50, 100))
cv2.waitKey(0)

-1

In [16]:
M = np.float32([[1, 0, -50], [0, 1, -90]])
shifted = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
cv2.imshow("Shifted Up and Left", shifted)
cv2.waitKey(0)

-1

In [17]:
cv2.imshow("Shifted Up and Left", shift_image(image, -50, -90))
cv2.waitKey(0)

-1

In [18]:
(h, w) = image.shape[:2]
center = (w // 2, h // 2)

In [19]:
def image_rotate(image, degrees):
    rads = math.radians(degrees)
    cx, cy = (image.shape[1]//2, image.shape[0]//2)
    height_rot_img = round(abs(image.shape[0]*math.sin(rads))) + round(abs(image.shape[1]*math.cos(rads)))
    width_rot_img = round(abs(image.shape[1]*math.cos(rads))) + round(abs(image.shape[0]*math.sin(rads)))
    rot_img = np.uint8(np.zeros((height_rot_img,width_rot_img,image.shape[2])))
    midx,midy = (width_rot_img//2, height_rot_img//2)
    for i in range(rot_img.shape[0]):
        for j in range(rot_img.shape[1]):
            x= (i-midx)*math.cos(rads)+(j-midy)*math.sin(rads)
            y= -(i-midx)*math.sin(rads)+(j-midy)*math.cos(rads)
            x=round(x)+cy
            y=round(y)+cx
            if (x>=0 and y>=0 and x<image.shape[0] and  y<image.shape[1]):
                rot_img[i,j,:] = image[x,y,:]
    return rot_img 


In [20]:
cv2.imshow("Rotated by -90 Degrees", image_rotate(image, -90))
cv2.waitKey(0)

-1

In [21]:
M = cv2.getRotationMatrix2D(center, -90, 1.0)
rotated = cv2.warpAffine(image, M, (w, h))
cv2.imshow("Rotated by -90 Degrees", rotated)
cv2.waitKey(0)

-1

In [22]:
cv2.imshow("Rotated by 45 Degrees", image_rotate(image, 45))
cv2.waitKey(0)

-1

In [23]:
M = cv2.getRotationMatrix2D(center, 45, 1.0)
rotated = cv2.warpAffine(image, M, (w, h))
cv2.imshow("Rotated by 45 Degrees", rotated)
cv2.waitKey(0)

-1

In [24]:
def resize_img(image, resize_width, resize_height):   
    original_width, original_height, channel = image.shape
    
    red_channel   = image[:, :, 0]
    green_channel = image[:, :, 1]
    blue_channel  = image[:, :, 2]
    
    resized_image = np.zeros((resize_width, resize_height, channel), dtype=np.uint8)
    
    x_scale = original_width/resize_width
    y_scale = original_height/resize_height
    
    resize_idx = np.zeros((resize_width, resize_height))
    resize_index_x = np.ceil(np.arange(0, original_width, x_scale)).astype(int)
    resize_index_y = np.ceil(np.arange(0, original_height, y_scale)).astype(int)
    resize_index_x[np.where(resize_index_x == original_width)]  -= 1
    resize_index_y[np.where(resize_index_y == original_height)] -= 1

    resized_image[:, :, 0] = red_channel[resize_index_x, :][:, resize_index_y]
    resized_image[:, :, 1] = green_channel[resize_index_x, :][:, resize_index_y]
    resized_image[:, :, 2] = blue_channel[resize_index_x, :][:, resize_index_y]
    
    return resized_image

In [25]:
r = 150.0 / image.shape[1]
dim = (150, int(image.shape[0] * r))
cv2.imshow("Resized (Width)", resize_img(image,dim[0],dim[1]))
cv2.waitKey(0)

-1

In [26]:
r = 150.0 / image.shape[1]
dim = (150, int(image.shape[0] * r))

In [27]:
resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
cv2.imshow("Resized (Width)", resized)
cv2.waitKey(0)

-1

In [28]:
r = 50.0 / image.shape[0]
dim = (int(image.shape[1] * r), 50)
resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
cv2.imshow("Resized (Height)", resized)
cv2.waitKey(0)

-1

In [55]:
r = 50.0 / image.shape[0]
dim = (int(image.shape[1] * r), 50)
cv2.imshow("Resized (Height)", resize_img(image,dim[0],dim[1]))
cv2.waitKey(0)

-1

In [29]:
flipped = cv2.flip(image, 1)
cv2.imshow("Flipped Horizontally", flipped)
cv2.waitKey(0)

-1

In [30]:
def flip(image,flip):
    if flip >= 1:
        h_flip = image[:, ::-1]
        return h_flip
    elif flip == 0: 
        v_flip = image[::-1]
        return v_flip
    else:
        h_flip = image[:, ::-1]
        v_flip = h_flip[::-1]
        return v_flip
# flip horizontally
    

In [31]:
cv2.imshow("Flipped Horizontally", flip(image,1))
cv2.waitKey(0)

-1

In [32]:
def crop_img(image,width,heigt,x,y):
    cropped = image[x:x+width , y:y+heigt]
    return cropped


In [33]:
crop = image[100:200, 500:600]
cv2.imshow("T-Rex Face", crop)
cv2.waitKey(0)

-1

In [34]:
cv2.imshow("T-Rex Face", crop_img(image,100,100,100,500))
cv2.waitKey(0)

-1

In [35]:
M = np.ones(image.shape, dtype = "uint8") * 100
added = cv2.add(image, M)
cv2.imshow("Added", added)
cv2.waitKey(0)

-1

In [36]:
def image_add_matrix(image,filters):
    result = np.zeros(image.shape,dtype='uint8')
    for i in range(image.shape[0]):  
        for j in range(image.shape[1]):
            if float(image[i,j,0]) + float(filters[i,j,0]) >= 255:
                result[i,j,0] = 255
            else:
                result[i,j,0] = float(image[i,j,0]) + float(filters[i,j,0])
            if float(image[i,j,1]) + float(filters[i,j,1]) >= 255:
                result[i,j,1] = 255
            else:
                result[i,j,1] = float(image[i,j,1]) + float(filters[i,j,1])
            if float(image[i,j,2]) + float(filters[i,j,2]) >= 255:
                result[i,j,2] = 255
            else:
                result[i,j,2] = float(image[i,j,2]) + float(filters[i,j,2])
    return result

In [37]:
M = np.ones(image.shape, dtype = "uint8")*100
cv2.imshow("Added", image_add_matrix(image,M))
cv2.waitKey(0)

-1

In [38]:
def image_substract_matrix(image,filters):
    result = np.zeros(image.shape,dtype='uint8')
    for i in range(image.shape[0]):  
        for j in range(image.shape[1]):
            if float(image[i,j,0]) - float(filters[i,j,0]) <= 0:
                result[i,j,0] = 0
            else:
                result[i,j,0] = float(image[i,j,0]) - float(filters[i,j,0])
            if float(image[i,j,1]) - float(filters[i,j,1]) <= 0:
                result[i,j,1] = 0
            else:
                result[i,j,1] = float(image[i,j,1]) - float(filters[i,j,1])
            if float(image[i,j,2]) - float(filters[i,j,2]) <= 0:
                result[i,j,2] = 0
            else:
                result[i,j,2] = float(image[i,j,2]) - float(filters[i,j,2])
    return result

In [39]:
M = np.ones(image.shape, dtype = "uint8") * 100
cv2.imshow("Subtracted", image_substract_matrix(image,M))
cv2.waitKey(0)

-1

In [40]:
M = np.ones(image.shape, dtype = "uint8") * 100
subtracted = cv2.subtract(image, M)
cv2.imshow("Subtracted", subtracted)
cv2.waitKey(0)

-1

In [41]:
rectangle = np.zeros((300, 300), dtype = "uint8")
cv2.rectangle(rectangle, (25, 25), (275, 275), 255, -1)
cv2.imshow("Rectangle", rectangle)
cv2.waitKey(0)

-1

In [42]:
circle = np.zeros((300, 300), dtype = "uint8")
cv2.circle(circle, (150, 150), 150, 255, -1)
cv2.imshow("Circle", circle)
cv2.waitKey(0)

-1

In [43]:
bitwiseAnd = cv2.bitwise_and(rectangle, circle)
cv2.imshow("AND", bitwiseAnd)
cv2.waitKey(0)

-1

In [44]:
def bitwise_and(image,mask):
    canvas = np.zeros(image.shape,dtype='uint8')
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if len(image.shape) == 3:
                for k in range(image.shape[2]):
                    canvas[i,j,k] = image[i,j,k] & mask[i,j]
            canvas[i,j] = image[i,j] & mask[i,j]
    return canvas

In [45]:
bitwiseAnd = bitwise_and(rectangle, circle)
cv2.imshow("AND", bitwiseAnd)
cv2.waitKey(0)

-1

In [46]:
bitwiseOr = cv2.bitwise_or(rectangle, circle)
cv2.imshow("OR", bitwiseOr)
cv2.waitKey(0)

-1

In [47]:
def bitwise_or(image,mask):
    canvas = np.zeros(image.shape,dtype='uint8')
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if len(image.shape) == 3:
                for k in range(image.shape[2]):
                    canvas[i,j,k] = image[i,j,k] | mask[i,j,k]
            canvas[i,j] = image[i,j] | mask[i,j]
    return canvas

In [48]:
bitwiseOr = bitwise_or(rectangle, circle)
cv2.imshow("OR", bitwiseOr)
cv2.waitKey(0)

-1

In [49]:
bitwiseXor = cv2.bitwise_xor(rectangle, circle)
cv2.imshow("XOR", bitwiseXor)
cv2.waitKey(0)

-1

In [50]:
def bitwise_xor(image,mask):
    canvas = np.zeros(image.shape,dtype='uint8')
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if len(image.shape) == 3:
                for k in range(image.shape[2]):
                    canvas[i,j,k] = image[i,j,k] ^ mask[i,j,k]
            canvas[i,j] = image[i,j] ^ mask[i,j]
    return canvas

In [51]:
bitwiseXor = bitwise_xor(rectangle, circle)
cv2.imshow("XOR", bitwiseXor)
cv2.waitKey(0)

-1

In [52]:
bitwiseNot = cv2.bitwise_not(circle)
cv2.imshow("NOT", bitwiseNot)
cv2.waitKey(0)

-1

In [53]:
def bitwise_not(image):
    canvas = np.zeros(image.shape,dtype='uint8')
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if len(image.shape) == 3:
                for k in range(image.shape[2]):
                    canvas[i,j,k] = ~image[i,j,k]
            canvas[i,j] = ~image[i,j]
    return canvas

In [54]:
bitwiseNot = bitwise_not(circle)
cv2.imshow("NOT", bitwiseNot)
cv2.waitKey(0)

-1

In [55]:
(cX, cY) = (image.shape[1] // 2, image.shape[0] // 2)
mask = np.zeros(image.shape[:2], dtype = "uint8")
circle = cv2.circle(mask, (cX, cY), 100, 255, -1)

In [56]:
circle.shape

(451, 697)

In [57]:
mask = bitwise_and(image,circle)    
cv2.imshow("mask", mask)
cv2.waitKey(0)

-1

In [70]:

(B, G, R) = cv2.split(image)

In [71]:
cv2.imshow("Red", R)
cv2.imshow("Green", G)
cv2.imshow("Blue", B)
cv2.waitKey(0)

-1

In [8]:
def splitting_channels(image):
    return image[:,:,0],image[:,:,1],image[:,:,2]

In [72]:
(blue,green,red) = splitting_channels(image)
cv2.imshow("Red", red)
cv2.imshow("Green", green)
cv2.imshow("Blue", blue)
cv2.waitKey(0)

-1

In [76]:
merged = cv2.merge([B, G, R])
cv2.imshow("Merged", merged)
cv2.waitKey(0)

-1

In [15]:
def merging_channels(r,g,b):
    image = np.zeros((r.shape[0],r.shape[1],3),dtype=np.uint8)
    image[:,:,0] = r
    image[:,:,1] = g
    image[:,:,2] = b
    return image

In [75]:
cv2.imshow("Merged", merging_channels(red,green,blue))
cv2.waitKey(0)

-1

In [77]:
zeros = np.zeros(image.shape[:2], dtype = "uint8")
cv2.imshow("Red", cv2.merge([zeros, zeros, R]))
cv2.imshow("Green", cv2.merge([zeros, G, zeros]))
cv2.imshow("Blue", cv2.merge([B, zeros, zeros]))
cv2.waitKey(0)

-1

In [79]:
zeros = np.zeros(image.shape[:2], dtype = "uint8")
cv2.imshow("Red", merging_channels(zeros, zeros, R))
cv2.imshow("Green", merging_channels(zeros, G, zeros))
cv2.imshow("Blue", merging_channels(B, zeros, zeros))
cv2.waitKey(0)

-1

In [81]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imshow("Gray", gray)
cv2.waitKey(0)

-1

In [80]:
def rgb2gray(image):
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            image[i][j]=max(image[i][j][0],image[i][j][1],image[i][j][2])
    return image

In [82]:
cv2.imshow("Gray", rgb2gray(image))
cv2.waitKey(0)

-1

In [60]:
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
cv2.imshow("HSV", hsv)
cv2.waitKey(0)

-1

In [58]:
def rgb2hsv(image):
    hsv = np.zeros(image.shape,dtype='uint8')
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            r = image[i,j,0]
            g = image[i,j,1]
            b = image[i,j,2]
            cmax = max(r,g,b)
            cmin = min(r,g,b)
            delta = cmax - cmin
            if delta == 0:
                h = 0
            elif cmax == r:
                h = 60*(((g-b)/delta)%6)
            elif cmax == g:
                h = 60*(((g-b)/delta)%2)
            else:
                h = 60*(((g-b)/delta)%4)
            if cmax == 0:
                s = 0
            else:
                s = delta/cmax
            v = cmax
            hsv[i,j,0] = h
            hsv[i,j,1] = s 
            hsv[i,j,2] = v
    return hsv

In [59]:
hsv = rgb2hsv(image)
cv2.imshow("HSV", hsv)
cv2.waitKey(0)

-1

In [61]:
def rgb2lab(image):
    lab = np.zeros(image.shape,dtype='float64')
    image = image.astype('float64')
    # image = image/255
    # image = np.where(image>0.04045,((image+0.055)/1.055)**2.4,image/12.92)
    # image = image*100
    for i in range (image.shape[0]):
        for j in range(image.shape[1]):
            image[i,j] = image[i,j]/255
            x = image[i,j,0]*0.412453 + image[i,j,1]*0.35758 + image[i,j,2]*0.180423
            y = image[i,j,0]*0.212671 + image[i,j,1]*0.71516 + image[i,j,2]*0.072169
            z = image[i,j,0]*0.019334 + image[i,j,1]*0.119193 + image[i,j,2]*0.950227
            x = x/95.047
            y = y/100
            z = z/108.883
            x = np.where(x>0.0085,x**(1/3),(7.787 *x)+16/116)
            y = np.where(y>0.0085,y**(1/3),(7.787 *y)+16/116)
            z = np.where(z>0.0085,z**(1/3),(7.787 *z)+16/116)
            lab[i,j,0] = ((116*y)-16)/10
            lab[i,j,1] = (500*(x-y))/10
            lab[i,j,2] = (200*(y-z))/5
    return lab
    

In [62]:
lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
cv2.imshow("L*a*b*", lab)
cv2.waitKey(0)

-1

In [63]:
cv2.imshow("My L*a*b*", rgb2lab(image))
cv2.waitKey(0)

-1